In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

try:
    df = pd.read_csv('/content/job_data_Careerviet_processed (5).csv')
    print("Tải dữ liệu đã qua xử lý văn bản thành công.")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'job_data_final_processed_desc_req.csv'.")
    print("Vui lòng đảm bảo bạn đã chạy code tiền xử lý văn bản trước.")


df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
df['feature_text'] = df['Description'].fillna('') + ' ' + df['Requirement'].fillna('')

print(f"\nTổng số dòng trong dữ liệu: {len(df)}")

bad_date_condition = (df['Time'] == pd.to_datetime('1970-01-01'))
df_good_time = df[~bad_date_condition].copy()
df_bad_time = df[bad_date_condition].copy()

print(f"Số dòng có ngày hợp lệ: {len(df_good_time)}")
print(f"Số dòng có ngày cần sửa (1/1/1970): {len(df_bad_time)}")


if not df_bad_time.empty and not df_good_time.empty:
    print("\nBắt đầu quá trình điền ngày tháng bị thiếu...")

    # Vector hóa văn bản bằng TF-IDF
    vectorizer = TfidfVectorizer()
    # Fit và transform trên toàn bộ tập dữ liệu để có một bộ từ vựng chung
    tfidf_matrix = vectorizer.fit_transform(df['feature_text'])

    # Lấy các vector tương ứng với dữ liệu "tốt" và "xấu"
    tfidf_good = tfidf_matrix[df_good_time.index]
    tfidf_bad = tfidf_matrix[df_bad_time.index]

    # Tính toán độ tương đồng cosine giữa mỗi dòng "xấu" và tất cả các dòng "tốt"
    print("Đang tính toán độ tương đồng...")
    similarity_matrix = cosine_similarity(tfidf_bad, tfidf_good)

    # Tìm index của dòng "tốt" giống nhất cho mỗi dòng "xấu"
    # np.argmax sẽ trả về vị trí (index) của giá trị lớn nhất trong mỗi hàng
    most_similar_indices = np.argmax(similarity_matrix, axis=1)

    # Lấy ra các giá trị thời gian từ các dòng "tốt" tương ứng
    imputed_times = df_good_time.iloc[most_similar_indices]['Time'].values

    # Điền các giá trị thời gian đã tìm được vào DataFrame gốc
    df.loc[df_bad_time.index, 'Time'] = imputed_times

    print("Hoàn tất việc điền ngày tháng.")
else:
    print("\nKhông có dòng nào cần sửa hoặc không có dữ liệu gốc để đối chiếu.")



remaining_bad_dates = (df['Time'] == pd.to_datetime('1970-01-01')).sum()
print(f"\nSố dòng còn lại có ngày 1/1/1970 sau khi xử lý: {remaining_bad_dates}")

# Xóa các cột xử lý trung gian
df.drop(columns=['feature_text'], inplace=True, errors='ignore')

print("\nCác cột cuối cùng trong DataFrame:")
print(df.info())

# Lưu DataFrame cuối cùng ra file mới
output_file = 'job_data_final_imputed.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\nĐã lưu thành công dữ liệu đã được sửa vào file: {output_file}")


Tải dữ liệu đã qua xử lý văn bản thành công.

Tổng số dòng trong dữ liệu: 21861
Số dòng có ngày hợp lệ: 21105
Số dòng có ngày cần sửa (1/1/1970): 756

Bắt đầu quá trình điền ngày tháng bị thiếu...
Đang tính toán độ tương đồng...
Hoàn tất việc điền ngày tháng.

Số dòng còn lại có ngày 1/1/1970 sau khi xử lý: 0

Các cột cuối cùng trong DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21861 entries, 0 to 21860
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Job_Name      21861 non-null  object        
 1   Company_name  21861 non-null  object        
 2   Location      21861 non-null  object        
 3   Salary        21861 non-null  object        
 4   Time          21861 non-null  datetime64[ns]
 5   Benefit       21249 non-null  object        
 6   Description   21553 non-null  object        
 7   Requirement   21596 non-null  object        
 8   URL           21861 non-null  object    